In [ ]:
import os
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_learning_rate: float
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
os.chdir('../')

In [6]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories
import tensorflow as tf
# if not tf.executing_eagerly():
#     tf.config.run_functions_eagerly(True)

2025-07-09 12:33:11.259167: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-09 12:33:11.413044: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-09 12:33:11.528418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752044591.600829   70170 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752044591.622490   70170 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-09 12:33:11.788293: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [7]:
tf.compat.v1.enable_eager_execution()

In [ ]:
class ConfigurationManager:
    def __init__(
        self, config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([Path(training.root_dir)])
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_learning_rate = params.LEARNING_RATE,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [9]:
import urllib.request as request
from zipfile import ZipFile
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        
        # Loaded with compile=False, and then recompiled below to AVOID GRAPH MODE and loading the
        # saved model in Eager Mode only, so as to avoid numpy related conflicts
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params.LEARNING_RATE,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy']
        )
        

    def train_valid_generator(self):
        image_size = tuple(self.config.params_image_size[:-1])  # (height, width)
        batch_size = self.config.params_batch_size

        # Load datasets using the newer, eager-compatible API
        self.train_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="training",
            seed=123,
            image_size=image_size,
            batch_size=batch_size
        )

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="validation",
            seed=123,
            image_size=image_size,
            batch_size=batch_size
        )

        # Normalization layer (rescale pixel values from [0, 255] to [0, 1])
        normalization_layer = tf.keras.layers.Rescaling(1./255)

        self.train_generator = self.train_generator.map(lambda x, y: (normalization_layer(x), y))
        self.valid_generator = self.valid_generator.map(lambda x, y: (normalization_layer(x), y))

        # Optional augmentation if enabled
        if self.config.params_is_augmentation:
            augmentation = tf.keras.Sequential([
                tf.keras.layers.RandomFlip("horizontal"),
                tf.keras.layers.RandomRotation(0.1),
                tf.keras.layers.RandomZoom(0.2),
                tf.keras.layers.RandomTranslation(0.1, 0.1),
            ])
            self.train_generator = self.train_generator.map(lambda x, y: (augmentation(x), y))

        # Performance optimization
        AUTOTUNE = tf.data.AUTOTUNE
        self.train_generator = self.train_generator.prefetch(buffer_size=AUTOTUNE)
        self.valid_generator = self.valid_generator.prefetch(buffer_size=AUTOTUNE)


    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )  

In [15]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-07-09 12:50:08,300: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-09 12:50:08,306: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-09 12:50:08,308: INFO: common: created directory at: artifacts]
[2025-07-09 12:50:08,309: INFO: common: created directory at: artifacts/training]
Found 465 files belonging to 2 classes.
Using 372 files for training.
Found 465 files belonging to 2 classes.
Using 93 files for validation.
Eager execution is (before model.fit): True




2025-07-09 12:50:10.834025: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-07-09 12:50:11.315413: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-07-09 12:50:12.085282: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2025-07-09 12:50:12.140144: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2025-07-09 12:50:12.398442: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


24/24 ━━━━━━━━━━━━━━━━━━━━ 146s 6s/step - accuracy: 0.5837 - loss: 10.4205 - val_accuracy: 0.4946 - val_loss: 24.4841
[2025-07-09 12:52:35,202: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
